<a href="https://colab.research.google.com/github/AntioTseng/LSTM2/blob/main/%E6%9C%80%E4%BD%B3%E5%8C%96%E8%82%A1%E7%A5%A8%E4%BA%A4%E6%98%93%E7%AD%96%E7%95%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://hackmd.io/@s02260441/rJ3VNnQlv

### 利用5日MA線(週移動平均線)與15日MA線，作簡單的交易策略，
### 再利用迴圈取得各產業績效指數最高的幾名，進而找到績優股。

In [ ]:
#!pip install yfinance
#!pip install backtesting

In [ ]:
from backtesting import Backtest, Strategy #引入回測和交易策略功能

from backtesting.lib import crossover #從lib子模組引入判斷均線交會功能
from backtesting.test import SMA #從test子模組引入繪製均線功能

import pandas as pd #引入pandas讀取股價歷史資料CSV檔
import yfinance as yf
import pandas as pd
from pandas_datareader import data
from datetime import datetime
import matplotlib   
import matplotlib.pyplot as plt  # for 畫圖用
import datetime as dt

In [11]:
class SmaCross(Strategy): #交易策略命名為SmaClass，使用backtesting.py的Strategy功能
    n1 = 5 #設定第一條均線日數為5日(周線)
    n2 = 15 #設定第二條均線日數為20日(月線)，這邊的日數可自由調整

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1) #定義第一條均線為sma1，使用backtesting.py的SMA功能算繪
        self.sma2 = self.I(SMA, self.data.Close, self.n2) #定義第二條均線為sma2，使用backtesting.py的SMA功能算繪

    def next(self):
        if crossover(self.sma1, self.sma2): #如果周線衝上月線，表示近期是上漲的，則買入
            self.buy()
        elif crossover(self.sma2, self.sma1): #如果周線再與月線交叉，表示開始下跌了，則賣出
            self.sell()

In [12]:
firm = ["1903.TW", "1904.TW", "1905.TW", "1906.TW", "1907.TW", "1909.TW", "6790.TW", #造紙產業
        "1802.TW", "1806.TW", "1809.TW", "1810.TW", "1817.TW", #玻璃產業
        "2701.TW", "2702.TW", "2704.TW",	"2705.TW", "2706.TW",
        "2707.TW", "2712.TW", "2722.TW",	"2723.TW", "2727.TW",
        "2731.TW", "2739.TW", "2748.TW",	"2753.TW", "5706.TW",
        "8462.TW", "8940.TW", "9943.TW", #觀光產業
        "1342.TW", "1416.TW", "1435.TW", "1437.TW",
        "1443.TW", "1516.TW", "2062.TW", "2348.TW",
        "2348A.TW", "2358.TW", "2443.TW", "2496.TW",
        "2514.TW", "2614.TW", "2904.TW", "3040.TW",
        "3557.TW", "4536.TW", "5284.TW", "5306.TW",
        "5871.TW", "5871A.TW", "6165.TW", "6184.TW",
        "6464.TW", "6504.TW", "6581.TW", "6585.TW",
        "6592.TW", "6625.TW", "6641.TW", "6655.TW",
        "6670.TW", "6671.TW", "6754.TW", "6768.TW",
        "6806.TW", "6807.TW", "8033.TW", "8341.TW",
        "8404.TW", "8411.TW", "8422.TW", "8438.TW",
        "8442.TW", "8463.TW", "8464.TW", "8466.TW",
        "8467.TW", "8473.TW", "8478.TW", "8480.TW",
        "8481.TW", "8482.TW", "8488.TW", "9802.TW",
        "9902.TW", "9904.TW", "9905.TW", "9907.TW",
        "9910.TW", "9911.TW", "9914.TW", "9917.TW",
        "9919.TW", "9921.TW", "9924.TW", "9925.TW",
        "9927.TW", "9928.TW", "9929.TW", "9930.TW",
        "9933.TW", "9934.TW", "9935.TW", "9938.TW",
        "9939.TW", "9940.TW", "9941.TW", "9941A.TW",
        "9942.TW", "9944.TW", "9945.TW", "9955.TW", #其他產業
        "2002.TW","2002A.TW","2006.TW","2007.TW","2008.TW","2009.TW",
        "2010.TW","2012.TW","2013.TW","2014.TW","2015.TW","2017.TW",
        "2020.TW","2022.TW","2023.TW","2024.TW","2027.TW",
        "2028.TW","2029.TW","2030.TW","2031.TW","2032.TW","2033.TW",
        "2034.TW","2038.TW","2069.TW","2211.TW","3004.TW","5007.TW",
        "5538.TW","9958.TW"] #鋼鐵產業
SQN = []
Sharpe = []
Calmar = []
start_date = dt.datetime(2020, 1, 1)
end_date = dt.datetime(2022, 7, 18) #設定資料起訖日期

In [13]:
for i in range(len(firm)):
  yf.pdr_override() #以pandasreader常用的格式覆寫

  target_stock = firm[i]  #股票代號變數

  df = data.get_data_yahoo(target_stock, start_date, end_date) #將資料放到Dataframe裡面

  filename = f"%s.csv"%firm[i] #以股票名稱命名檔案，放在data資料夾下面

  df.to_csv(filename) #將df轉成CSV保存
  df = pd.read_csv(f"%s.csv"%firm[i], index_col=0) #pandas讀取資料，並將第1欄作為索引欄
  df = df.interpolate() #CSV檔案中若有缺漏，會使用內插法自動補值，不一定需要的功能
  df.index = pd.to_datetime(df.index) #將索引欄資料轉換成pandas的時間格式，backtesting才有辦法排序

  test = Backtest(df, SmaCross, cash=10000, commission=.002)
  # 指定回測程式為test，在Backtest函數中依序放入(資料來源、策略、現金、手續費)
  opt_result_sqn = test.optimize(n1=range(5, 50, 5),  #將回測機器加入optimize屬性，定義短均線的周期為5~50(每次加5)，長均線的周期為10~120(每次加5)
                      n2=range(10, 120, 5),
                      maximize='SQN',  #最佳化目標為SQN最大化
                      constraint=lambda p: p.n1 < p.n2)  #限制n1及n2的範圍，只會計算n1小於n2的情況
        
  print() #空一行
  print("Optimize strategy")
  print(opt_result_sqn)  #印出opt_result_sqn結果

  Sharpe.append(opt_result_sqn['Sharpe Ratio'])
  Calmar.append(opt_result_sqn['Calmar Ratio'])
  SQN.append(opt_result_sqn['SQN'])

[*********************100%***********************]  1 of 1 completed


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]


Optimize strategy
Start                     2020-01-02 00:00:00
End                       2022-07-15 00:00:00
Duration                    925 days 00:00:00
Exposure Time [%]                   68.506494
Equity Final [$]                 12810.940054
Equity Peak [$]                  13140.842258
Return [%]                          28.109401
Buy & Hold Return [%]                62.79069
Return (Ann.) [%]                   10.665031
Volatility (Ann.) [%]               19.285305
Sharpe Ratio                         0.553013
Sortino Ratio                        0.838092
Calmar Ratio                         0.531163
Max. Drawdown [%]                  -20.078622
Avg. Drawdown [%]                   -4.252792
Max. Drawdown Duration      529 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                      20.042109
Worst Trade [%]                     13.285102
Avg. Trade [%] 

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]


Optimize strategy
Start                     2020-01-02 00:00:00
End                       2022-07-15 00:00:00
Duration                    925 days 00:00:00
Exposure Time [%]                   61.525974
Equity Final [$]                 12156.734066
Equity Peak [$]                  12388.783929
Return [%]                          21.567341
Buy & Hold Return [%]               30.848328
Return (Ann.) [%]                    8.317301
Volatility (Ann.) [%]               60.921125
Sharpe Ratio                         0.136526
Sortino Ratio                        0.226028
Calmar Ratio                         0.127843
Max. Drawdown [%]                  -65.058937
Avg. Drawdown [%]                  -15.238995
Max. Drawdown Duration      515 days 00:00:00
Avg. Drawdown Duration      112 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      19.559617
Worst Trade [%]                     14.515496
Avg. Trade [%] 

...


Optimize strategy
Start                     2020-01-02 00:00:00
End                       2022-07-15 00:00:00
Duration                    925 days 00:00:00
Exposure Time [%]                   98.051948
Equity Final [$]                  10609.69345
Equity Peak [$]                  12637.693748
Return [%]                           6.096935
Buy & Hold Return [%]                2.272729
Return (Ann.) [%]                    2.450668
Volatility (Ann.) [%]               17.114582
Sharpe Ratio                         0.143192
Sortino Ratio                        0.206211
Calmar Ratio                         0.144386
Max. Drawdown [%]                  -16.973039
Avg. Drawdown [%]                     -4.9688
Max. Drawdown Duration      452 days 00:00:00
Avg. Drawdown Duration      109 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      19.190822
Worst Trade [%]                      6.037926
Avg. Trade [%] 

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]


Optimize strategy
Start                     2020-01-02 00:00:00
End                       2022-07-15 00:00:00
Duration                    925 days 00:00:00
Exposure Time [%]                   86.850649
Equity Final [$]                 13867.240123
Equity Peak [$]                  20672.440367
Return [%]                          38.672401
Buy & Hold Return [%]                33.70253
Return (Ann.) [%]                   14.310686
Volatility (Ann.) [%]               37.930636
Sharpe Ratio                         0.377286
Sortino Ratio                        0.710059
Calmar Ratio                         0.414207
Max. Drawdown [%]                  -34.549622
Avg. Drawdown [%]                   -8.143067
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                      53.977761
Worst Trade [%]                     38.629494
Avg. Trade [%] 

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]


Optimize strategy
Start                     2020-01-02 00:00:00
End                       2022-07-15 00:00:00
Duration                    925 days 00:00:00
Exposure Time [%]                   62.662338
Equity Final [$]                 12706.949168
Equity Peak [$]                  12706.949168
Return [%]                          27.069492
Buy & Hold Return [%]               30.027553
Return (Ann.) [%]                   10.296656
Volatility (Ann.) [%]               35.101911
Sharpe Ratio                         0.293336
Sortino Ratio                        0.472144
Calmar Ratio                         0.373237
Max. Drawdown [%]                  -27.587476
Avg. Drawdown [%]                   -8.910378
Max. Drawdown Duration      267 days 00:00:00
Avg. Drawdown Duration       52 days 00:00:00
# Trades                                    6
Win Rate [%]                            100.0
Best Trade [%]                      21.393809
Worst Trade [%]                      9.206167
Avg. Trade [%] 

In [15]:
output = pd.concat([pd.DataFrame(Sharpe,columns=['Sharpe']),pd.DataFrame(Calmar,columns=['Calmar']), pd.DataFrame(SQN,  columns = ['SQN'])],axis=1)
output.index = [firm]
output.sort_values(by=['SQN'],ascending=False).head(10)
#6806

,Sharpe,Calmar,SQN
1903.TW,0.553013,0.531163,59.933020
2723.TW,0.418801,0.440750,33.946719
6806.TW,0.751646,1.094193,29.672032
6165.TW,0.338856,0.391069,12.683551
6790.TW,1.330234,3.312176,9.160420
2017.TW,1.129927,2.569366,6.709591
2753.TW,0.734365,0.575440,5.486672
8463.TW,0.636928,0.650138,5.070121
9919.TW,0.812152,1.193018,5.043896
9929.TW,0.756279,1.500029,4.705603
